In [28]:
# Binary integer program (BIP) that assigns carry-in sites to prep sites
# This model does not consider delivery windows or vehicle capacity, those constraints will be considered when routing
# Input: supply of prep sites, demand of carry-in sites, cost of going from prep to carry-in site (time,distance,whatever)
# Output: pairings with carry-in sites listed first

from ortools.linear_solver import pywraplp
import pandas as pd 

In [29]:
## Model 
# Declare solver   
solver = pywraplp.Solver('SolveAssignmentProblemMIP',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [68]:
def convert_to_dict(keylist,valuelist):
    #makes two lists into a dictionary with the first list as the keys
    #list lengths must be equivalent
    temp_dict = {}
    if len(keylist) != len(valuelist):
        print("List lengths not equivalent")
        return
    for x in range(len(keylist)):
        temp_dict[keylist[x]] = valuelist[x]
    return temp_dict

def missing_key(master_dict,subset_dict):
    #returns keys that exist in master_dict but not in subset_dict
    #here master_dict will be segments and subset_dict will be costs
    #costs generated from current arcs but segments is list of all potential arcs
    #use to find missing keys if getting key error when running objective function cell
    missing_keys = {}
    for key in master_dict.keys():
        if key not in subset_dict.keys():
            missing_keys[key] = 1
    return missing_keys

def list_to_tuple(lst):
    # makes a list of lists into a list of tuples 
    # [[a,b],[c,d],[e,f],[g,h]] -> [(a,b),(c,d),(e,f),(g,h)]
    for i in range(len(lst)):
        lst[i] = tuple(lst[i])
    return lst

def amount_used():
## Dictionary list that has the estimated number of meals that each prep site will need to make
    for j in prep_site_number:
        used[j] = 0
        for i in carry_in_site_number:
            if segments[i,j].solution_value() > 0:
                used[j] = used[j] + carry_in_dict[i]
    return used

def get_list_of_pairings():
    pairings = [0]*len(assignments.keys())
    i = 0
    for key in assignments.keys():
        pairings[i] = key
        i = i + 1
    return pairings

In [31]:
## Make whole things function that takes in three file paths?

## Read in data
## change computer path
## Change column names in Excel file or here accordingly, remember to save Excel file before running again

# Supply
supply_df = pd.read_excel("C://Users/anyak/Documents/DPS/DPSData/ImportantDataCompiled.xlsx", "Prep Site Capacity",usecols = "B,C")
# pd.read_excel('filepath','worksheet in question', 'columns in question')
supply_df = supply_df.dropna().astype(int) # drop null values and convert data type to int
prep_site_number = supply_df["Prep Site #"].values.tolist() 
supply = supply_df["Supply"].values.tolist() # makes column of dataframe into list

# Demand
demand_df = pd.read_excel("C://Users/anyak/Documents/DPS/DPSData/ImportantDataCompiled.xlsx","Meals per School")
carry_in_site_number = demand_df["School #"].values.tolist() 
demand_df["Breakfast Demand"] = demand_df["Breakfast Demand"].astype(int)
demand_df["Lunch Demand"] = demand_df["Lunch Demand"].astype(int)
breakfast_demand = demand_df["Breakfast Demand"].dropna()
lunch_demand = demand_df["Lunch Demand"].dropna()
demand = breakfast_demand + lunch_demand
demand = demand.values.tolist()

# Distances
distances_df = pd.read_excel("C://Users/anyak/Documents/DPS/DPSData/preptocarry.xlsx",usecols = "D,E,F")
cost_list = distances_df["Distance(km)"].values.tolist()

# Destinations (carry ins) first for consistency going forward
arcs = distances_df[["Destination","Origin"]].values.tolist() ## list of lists, need to convert to list of tuples
arcs = list_to_tuple(arcs)

In [32]:
# turn lists into dictionaries
carry_in_dict = convert_to_dict(carry_in_site_number,demand)
prep_site_dict = convert_to_dict(prep_site_number,supply)
costs = convert_to_dict(arcs,cost_list)

In [33]:
## Declare binary decision variables
#i by j array segments where (i,j) is keyed to 1 if carry_in i is assigned to prep j and 0 otherwise
#segments is a dictionary which takes the tuple (i,j) as the key and the bool as the value
segments = {}
for i in carry_in_site_number:
    for j in prep_site_number:
        segments[(i, j)] = solver.BoolVar('seg[%i,%i]' % (i, j))

In [34]:
## Objective: minimize total distance between prep and carry sites 
# doesn't represent optimal minimization of drive time / miles driven, but ensures proximity heuristically
solver.Minimize(solver.Sum([costs[(i,j)]*segments[(i,j)]
                                        for i in carry_in_site_number
                                        for j in prep_site_number]))

In [35]:
## Constraints
# carry in i is assigned at least and no more than one prep j (prep j can have multiple carry_ins)
for i in carry_in_site_number:
    solver.Add(solver.Sum([segments[(i,j)] for j in prep_site_number]) == 1)

# sum of demand of carry_ins I assigned to j do not exceed supply of prep j
# carry_in_site_dict[i] gives demand of site i
# prep_site_dict[j] gives supply of prep j
for j in prep_site_number:
    solver.Add(solver.Sum([segments[(i,j)]*carry_in_dict[i] for i in carry_in_site_number]) <= prep_site_dict[j])

In [47]:
## Results
solver.Solve()
print('Total distance (km) = ', solver.Objective().Value())

#Prints assignments
assignments = {}
for i in carry_in_site_number:
    for j in prep_site_number:
        if segments[i, j].solution_value() > 0:
            assignments[(i,j)] = 1
            print('Carry in site %d assigned to prep site %d.' % (i,j))

Total distance (km) =  98.394440626
Carry in site 110 assigned to prep site 451.
Carry in site 117 assigned to prep site 419.
Carry in site 161 assigned to prep site 150.
Carry in site 168 assigned to prep site 455.
Carry in site 178 assigned to prep site 190.
Carry in site 179 assigned to prep site 424.
Carry in site 181 assigned to prep site 971.
Carry in site 192 assigned to prep site 248.
Carry in site 203 assigned to prep site 412.
Carry in site 212 assigned to prep site 423.
Carry in site 223 assigned to prep site 412.
Carry in site 252 assigned to prep site 190.
Carry in site 264 assigned to prep site 405.
Carry in site 316 assigned to prep site 982.
Carry in site 328 assigned to prep site 457.
Carry in site 383 assigned to prep site 455.
Carry in site 426 assigned to prep site 415.
Carry in site 436 assigned to prep site 464.
Carry in site 438 assigned to prep site 451.
Carry in site 473 assigned to prep site 218.
Carry in site 477 assigned to prep site 423.
Carry in site 478 a